# Chainerの学習済み物体検出モデルを利用した推論

## 概要
このノートブックでは、Chainerの学習済み物体検出モデルを利用して以下の2種類の推論を行います。

- 推論エンドポイントにモデルをホストして、リアルタイムに推論を実行する。
- リクエストを受けたときだけ、エンドポイントを起動し、S3にあるファイルを一括で推論する（バッチ変換ジョブ）。

物体検出のアルゴリズムはいくつかありますが、ここではSSD (Single Shot Multibox Detector) を利用します。Chainer公式の学習済みモデルは以下からダウンロードすることができます。

https://github.com/chainer/chainercv/tree/master/examples/ssd

### (注意点)
ChainerCVでは、特定のDeep Neural Networks(SSD含む)を構築する際に、学習済みモデルをロードすることができます。従って、*物体検出モデルの事前ダウンロードは不要*です。しかし今回は、学習済みモデルをSageMakerに取り込む方法を一通り体験するために、事前にダウンロードして、別途ロードするという手順を行います。

## 学習済みモデルのダウンロードとS3へのアップロード

上記のURLでは、SSD300とSSD512のモデルが提供されており、今回はSSD300を利用します。300や512は入力画像のサイズを表します。一般に、画像サイズの大きい512のほうが精度が良いですが、推論に多くの計算を必要とします。

SSD300のモデルをダウンロードするために以下を実行します。ダウンロードされたモデルは、このノートブックインスタンスの`/tmp/ssd_model.npz`に保存されます。SageMakerでモデルをホスティングするためには、tar.gz形式にしてS3にアップロードする必要があります。`model.tar.gz`に変換した後、SageMaker Python SDKの`upload_data`を利用してS3にアップロードします。アップロードされる先は、`s3://sagemaker-{リージョン名}-{12桁アカウントID}/notebook/chainercv_ssd/model.tar.gz`になります。

In [ ]:
import os
import shutil
import tarfile
import urllib.request

# Setup
from sagemaker import get_execution_role
import sagemaker
sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# Download the model weights.
try:
    url = 'https://chainercv-models.preferred.jp/ssd300_voc0712_trained_2017_08_08.npz'
    urllib.request.urlretrieve (url, '/tmp/ssd_model.npz')

# Tar and compress the model.
    with tarfile.open('/tmp/model.tar.gz', "w:gz") as tar:
         tar.add('/tmp/ssd_model.npz', arcname='ssd_model.npz')

# Upload the model. The `ChainerModel` will use `uploaded_data` to download this model.

    uploaded_model = sagemaker_session.upload_data(path='/tmp/model.tar.gz', 
                                                   key_prefix='notebook/chainercv_ssd')
finally:
    os.remove('/tmp/model.tar.gz')

## ホスティング用のスクリプトの作成

SageMakerでホスティングするためには、モデルの利用方法（どういうモデルを読み込むか、前処理をいれるかなど）を決めるPythonスクリプトが必要になります。Chainerの場合は、シリアライズされたモデルはネットワークの重みだけを含んでいるため、シンボルを定義して、そこに重みの値をロードするという処理が必要になります。

同梱されている`chainercv_ssd.py`が、そのためのスクリプトです。`chainercv_ssd.py`には、`model_fn`、`input_fn`、`predict_fn`の関数が定義されていることがわかります。以下は、ホスティングのために実装する関数の一覧です。このうち `model_fn` は必ず実装しなければなりません。


### ホスティング用関数一覧

* **`model_fn(model_dir)`**: この関数は`model_dir`に保存されているモデルをロードする関数です。上述したように、シンボルの定義を行ってからロードします。シンボルの定義にはChainerCVを利用します。

* **`input_fn(input_data, content_type)`**: この関数は、推論リクエストを受け付けたときに、推論用のデータ`input_data`に対する前処理を書く関数です。`content_type`を同時に受け取ることができるので、`content_type`に応じて条件分岐を作成し、異なる前処理を実装することができます。numpy形式`application/x-npy`を受け取る関数が標準実装されています。jpegなどのバイナリを受け取る場合は、その処理を追加実装する必要があります。
  
* **`predict_fn(input_data, model)`**: この関数は `input_fn` で前処理されてreturnされた値を`input_data`として受け取り、`model_fn`でロードした`model`で推論するコードを書く関数です。 
  
* **`output_fn(prediction, accept)`**: この関数は `predict_fn`のreturnした値`prediction`を後処理するための関数です。`accept`に応じて処理を変更することもできます。

### 関数の流れ
上記では文章で書きましたが、擬似的なコードで示すと、関数の実行順はこのような流れになります。
```python
# Load a model from file system
model = model_fn(model_dir)

# Deserialize the Invoke request body into an object we can perform prediction on
input_object = input_fn(request_body, request_content_type)

# Perform prediction on the deserialized object, with the loaded model
prediction = predict_fn(input_object, model)

# Serialize the prediction result into the desired response content type
output = output_fn(prediction, response_content_type)
```

## モデルをホスティング

S3にモデルをアップロードして、ホスティング用のスクリプトがそろったら、SageMakerにそのモデルを登録します。Chainerの場合は`ChainerModel`を利用します。このとき、アップロードしたモデルと、スクリプトを指定する必要があります。もしモデルを登録したら`deploy`でエンドポイントを作成します。

In [ ]:
from sagemaker.chainer.model import ChainerModel
from sagemaker.utils import sagemaker_timestamp

model = ChainerModel(model_data=uploaded_model, role=role, entry_point='chainercv_ssd.py')

endpoint_name = 'chainer-ssd-{}'.format(sagemaker_timestamp())

predictor = model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, endpoint_name=endpoint_name)

## ホストしたモデルに対して推論リクエストを送信

モデルをホストできたら`predictor`を利用して画像を送ります。`predictor.predict(image)`でリクエストを送ることができます。結果が、`predict_fn`で定義したように、bounding box, label, score がdict形式で返ってくることを確認しましょう。

In [ ]:
import chainercv
import numpy as np
from matplotlib import pyplot as plot

image = chainercv.utils.read_image('images/car1.jpg', color=True)
image = np.ascontiguousarray(image, dtype=np.uint8)

In [ ]:
result = predictor.predict(image).tolist()
bbox = result['bbox']
label = result['label']
score = result['score']

print('bounding box: {}\nlabel: {}\nscore: {}'.format(bbox, label, score))

ChainerCVを利用すると、簡単に画像の上にbounding box、label、scoreをのせることができます。

In [ ]:
%matplotlib inline
from chainercv.visualizations import vis_bbox
from chainercv.datasets import voc_bbox_label_names
import matplotlib.pyplot as plt
vis_bbox(image, bbox, label, score, label_names=voc_bbox_label_names)
plt.show()

## S3にあるファイルを一括で推論する（バッチ変換ジョブ）

エンドポイントを作成すると、リクエストに対してすぐに結果を返すことができます。一方で、エンドポイント実行中は常にコストがかかります。例えば、画像をS3に保存しておき、1日に1回まとめて推論を行う場合には、推論したいときだけエンドポイントを利用し、それ以外のときはエンドポイントを停止するほうが効率的です。それを実現するジョブが、バッチ変換ジョブです。

### バッチ変換ジョブの流れ

バッチ変換ジョブをリクエストすると以下のような処理がはしります。

1. エンドポイントを起動
1. 指定されたS3のファイル群をエンドポイントに送信
1. 推論結果をS3に保存
1. すべての推論が終わるとエンドポイントを削除

エンドポイントの削除まで自動で行われるため、コストを抑えながら、一括でデータを処理するのに向いています。
バッチ変換ジョブ用に`images`フォルダにおいている画像をS3にアップロードしましょう。アップロード先は
`s3://sagemaker-{リージョン名}-{12桁アカウントID}/batch_transform/ssd`です。

In [ ]:
image_set = sagemaker_session.upload_data(path='./images', key_prefix='batch_transform/ssd')
print("Images are uploaded to {}".format(image_set))

## バッチ変換ジョブの実行

バッチ変換ジョブを実行するためには、推論に利用するモデル`model`から`transformer`を作成します。`transoformer`に対して、アップロードした画像のS3のパスを渡して、バッチ変換ジョブを実行します。 

バッチ変換ジョブは大量のデータを一括処理するために用いられるので、実行してもノートブック側でその終了を待たない仕様になっています。ここでは、ジョブの終了がわかりやすいように、`tranformer.wait()`でジョブの終了を待ちます。

In [ ]:
transformer = model.transformer(instance_count=1,
                                output_path='s3://' +sagemaker_session.default_bucket() + '/batch_transform/output',
                                instance_type='ml.m4.xlarge')
transformer.transform(image_set, content_type='image/jpeg')

In [ ]:
transformer.wait()

## 出力結果をダウンロードして確認

出力先のS3パスが`transformer.output_path`に記録されているので、ここからファイルをダウンロードします。ダウンロードには、AWSのPython SDKである`boto3`を使います。出力ファイル名は`入力ファイル名.out`です。例えば、`car1.jpg`の出力結果は`car1.jpg.out`になります。ファイルの中身は`predict_fn`で定義した内容になります。

### 出力結果のダウンロード

ディレクトリ`output`に推論結果をダウンロードします。

In [ ]:
import json
from urllib.parse import urlparse

import boto3

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
prefix = parsed_url.path[1:]

s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket(bucket_name)

output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)
for key in bucket.objects.filter(Prefix = prefix):
    filename = os.path.basename(key.key)
    print("File {} is downloaded to {}".format(filename,output_dir))
    bucket.download_file(key.key, output_dir + "/" + filename)

### 出力結果の確認

ダウンロードした推論結果をパースして、画像の上に結果を出力します。dict形式で出力したので、`json.load`でファイルを読み込むことができます。

In [ ]:
%matplotlib inline
from chainercv.visualizations import vis_bbox
from chainercv.datasets import voc_bbox_label_names
import matplotlib.pyplot as plt

file_names = []
bbox_list = []
label_list = []
score_list = []

import json
for f in os.listdir(output_dir):
    if "out" in f:
        file_names.append(os.path.splitext(f)) 
        f = open("output/" + f, 'r')
        result = json.load(f)
        bbox_list.append(result['bbox'])
        label_list.append(result['label'])
        score_list.append(result['score'])

for i in range(len(file_names)):
    image = chainercv.utils.read_image("images/" + file_names[i][0], color=True)
    image = np.ascontiguousarray(image, dtype=np.uint8)
    vis_bbox(image, bbox_list[i], label_list[i], score_list[i], label_names=voc_bbox_label_names)
    plt.show()

## エンドポイントの削除

エンドポイントを起動しているとコストがかかり続けます。不要な場合は`delete_endpoint()`を呼び出して削除します。バッチ変換ジョブは、ジョブ終了に伴って自動でエンドポイントが削除されるため、ここで削除するのは、最初に`deploy`で作成したエンドポイントのみです。

In [ ]:
predictor.delete_endpoint()